author:leezeeyee   
date:2021/5/12

In [67]:
import numpy as np
from utils import timer

## Gram-Schmidt process

In [9]:
def proj(U,V):
    '''
    computer the projection of vector $V$ on the direction of $U$ 
    U and V should be same dimension
    '''
    if U.shape != V.shape:
        raise Exception('not in same dimension')
    return np.dot(U,V)*U/np.dot(U,U)

In [11]:
proj(np.array([3,4]),np.array([3,2]))

array([2.04, 2.72])

In [128]:
def gram_schmidt(A):
    U=np.copy(A)
    for col in range(A.shape[1]):
        #print(A[:,col]) 
        for i in range(col):
            # now the u is already guiyi
            U[:,col]-=np.dot(U[:,i],A[:,col])*U[:,i]#proj(U[:,i],A[:,col]) 
        U[:,col]/=np.sqrt(np.dot(U[:,col],U[:,col]))
    return U
            


In [51]:
res=gram_schmidt(np.random.rand(5,3))

In [52]:
res

array([[ 0.11989545,  0.06535308,  0.74256348],
       [ 0.62500627,  0.69793329,  0.06400778],
       [ 0.46046477, -0.05589415, -0.60180545],
       [ 0.49599204, -0.67357036,  0.09707424],
       [ 0.37007613, -0.22758937,  0.27001718]])

In [53]:
for col in range(res.shape[1]):
    print(np.dot(res[:,col],res[:,2]))

-4.0245584642661925e-16
4.163336342344337e-16
1.0


## QR decomposition

In [121]:
A=np.random.rand(3,3)
Q=gram_schmidt(A)
R=np.zeros([A.shape[1],A.shape[1]])
for j in range(A.shape[1]):
    for i in range(j+1):
        R[i][j]=np.dot(Q[:,i],A[:,j])


In [122]:
np.allclose(np.dot(Q,R),A)

True

In [71]:
@timer
def QR(A):
    Q=gram_schmidt(A)
    R=np.zeros([A.shape[1],A.shape[1]])
    for j in range(A.shape[1]):
        for i in range(j+1):
            R[i][j]=np.dot(Q[:,i],A[:,j])
    return Q,R

In [90]:
@timer
def inner(A):
    Q,R=np.linalg.qr(A)
    return Q,R

In [84]:
A=np.random.rand(10,49)

In [91]:
res,time=QR(A)
Q,R=res
print(time)

0:00:00.028005


In [92]:
res,time=inner(A)
print(time)


0:00:00.002168


## QR algorithm

In [147]:
def QR_eig(A):
    for i in range(1000):
        Q,R=QR.__wrapped__(A)
        A=np.dot(np.dot(Q.transpose(),A),Q)
    return A
    

In [152]:
d=3
A=np.random.rand(d,d)*10

In [153]:
A

array([[2.85556872, 2.24048991, 9.43285451],
       [8.40434956, 1.59401674, 5.50906956],
       [1.38480032, 0.57937951, 7.06994132]])

In [154]:
print(np.allclose(np.sort(np.linalg.eigvals(A)),np.sort(np.diag(QR_eig(A)))))

True


In [104]:
Q

matrix([[-1.,  0.],
        [-0.,  1.]])

In [106]:
Q.transpose()

matrix([[-1., -0.],
        [ 0.,  1.]])

In [107]:
np.dot(Q.transpose(),A)

matrix([[-4.0e+000,  1.0e+000],
        [-1.5e-323, -1.0e+000]])

In [108]:
np.dot(np.dot(Q.transpose(),A),Q)

matrix([[ 4.0e+000,  1.0e+000],
        [ 1.5e-323, -1.0e+000]])